In [ ]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#import the neccessary module
from helper import Model
from helper import Auxiliary

# Modelling
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session

In [78]:
#df_train, df_test = Auxiliary.train_test_split(Auxiliary.preprocess_data(Model.get_csv('BPI_Challenge_2012.csv')))
#read the data set
df_train = Auxiliary.preprocess_data(Model.get_csv('BPI_Challenge_2012_train.csv'))
df_test = Auxiliary.preprocess_data(Model.get_csv('BPI_Challenge_2012_test.csv'))

Start preprocessing data
Finished preprocessing data
Start preprocessing data
Finished preprocessing data


In [79]:
print(df_train.columns())


   org:resource  lifecycle:transition  concept:name  \
0         112.0                     2             0   
1         112.0                     2             1   
2         112.0                     2             2   
3         112.0                     0             3   
4           NaN                     1             3   

                     time:timestamp                     case:REG_DATE  \
0  2011-09-30 22:38:44.546000+00:00  2011-09-30 22:38:44.546000+00:00   
1  2011-09-30 22:38:44.880000+00:00  2011-09-30 22:38:44.546000+00:00   
2  2011-09-30 22:39:37.906000+00:00  2011-09-30 22:38:44.546000+00:00   
3  2011-09-30 22:39:38.875000+00:00  2011-09-30 22:38:44.546000+00:00   
4  2011-10-01 09:36:46.437000+00:00  2011-09-30 22:38:44.546000+00:00   

   case:concept:name  case:AMOUNT_REQ  A_SUBMITTED  A_PARTLYSUBMITTED  \
0             173688            20000            1                  0   
1             173688            20000            0                  1   
2          

In [ ]:

# encode the concept:name as integer values
df_encoded_train = df_train.copy()

lengthOfDf = len(df_encoded_train.index)

#add a new column with the position of each case
df_encoded_train.at[0, 'position'] = 1
j = 2
for i in range(1, lengthOfDf):
    if df_encoded_train.at[i, 'case:concept:name'] == df_encoded_train.at[i - 1, 'case:concept:name']:
        df_encoded_train.at[i, 'position'] = j
        j = j + 1
    else :
        df_encoded_train.at[i, 'position'] = 1
        j = 2


#the mode of the activities at a certain position
df_mode = df_encoded_train.groupby('position')['concept:name'].agg(pd.Series.mode)

#add a new column with the predicted time of each case and the ground truth
for i in range(lengthOfDf - 1):
    if df_encoded_train.at[i, 'position'] + 1 <= len(df_mode.index):
        if (df_encoded_train.at[i + 1, 'position']) == 1:
            df_encoded_train.at[i, 'Next time'] = 0
        else:
            df_encoded_train.at[i, 'Next time'] = Model.get_time_difference_as_number(df_encoded_train.at[i, 'time:timestamp'], df_encoded_train.at[i + 1, 'time:timestamp'])


In [ ]:

# encode the concept:name as integer values
df_encoded_test = df_test.copy()
lengthOfDf = len(df_encoded_test.index)

#add a new column with the position of each case
df_encoded_test.at[0, 'position'] = 1
j = 2
for i in range(1, lengthOfDf):
    if df_encoded_test.at[i, 'case:concept:name'] == df_encoded_test.at[i - 1, 'case:concept:name']:
        df_encoded_test.at[i, 'position'] = j
        j = j + 1
    else :
        df_encoded_test.at[i, 'position'] = 1
        j = 2


#the mode of the activities at a certain position
df_mode = df_encoded_test.groupby('position')['concept:name'].agg(pd.Series.mode)

#add a new column with the predicted time of each case and the ground truth
for i in range(lengthOfDf - 1):
    if df_encoded_test.at[i, 'position'] + 1 <= len(df_mode.index):
        if (df_encoded_test.at[i + 1, 'position']) == 1:
            df_encoded_test.at[i, 'Next time'] = 0
        else:
            df_encoded_test.at[i, 'Next time'] = Model.get_time_difference_as_number(df_encoded_test.at[i, 'time:timestamp'], df_encoded_test.at[i + 1, 'time:timestamp'])

In [ ]:

data_to_drop = ['org:resource', 'lifecycle:transition', 'case:AMOUNT_REQ', 'Next Event', 'case:REG_DATE', 'concept:name', 'case:concept:name', 'time:timestamp', 'position', 'minute',
       'second','year', 'month', 'hour']

#split the data into training and test sets and drop some data
x_train = df_encoded_train.drop(data_to_drop, axis=1)
x_train = x_train.drop(['W_Wijzigen contractgegevens'], axis = 1)
x_test = df_encoded_test.drop(data_to_drop, axis=1)
x_train = x_train.drop(['Next time'], axis=1)
x_test = x_test.drop(['Next time'], axis=1)

y_train = df_encoded_train['Next time']
y_test = df_encoded_test['Next time']

#fill all empty values with nan
x_train = x_train.replace(r'^\s*$', np.nan, regex=True)
x_test = x_test.replace(r'^\s*$', np.nan, regex=True)
y_train = y_train.replace(r'^\s*$', np.nan, regex=True)
y_test = y_test.replace(r'^\s*$', np.nan, regex=True)


#Fill all nan values with 0
x_train = x_train.replace(np.nan,0)
x_test = x_test.replace(np.nan,0)
y_train = y_train.replace(np.nan,0)
y_test = y_test.replace(np.nan,0)



#TODO x_train,x_test 


In [ ]:
print(x_test.head(10))
print(y_test.head(10))

In [ ]:
print(type(x_test))
print(x_test)

In [ ]:

from torch import from_numpy
#now everything is in array format
train_x = x_train
train_y = y_train
test_x = x_test
test_y = y_test



In [ ]:
final_train_x_list = []
final_test_x_list = []
for row in train_x.values:
    entry_list = []
    for value in row:
        entry_list.append(value)
    final_train_x_list.append(entry_list)
entry_list = []
for row in test_x.values:
    entry_list = []
    for value in row:
        entry_list.append(value)
    final_test_x_list.append(entry_list)

In [ ]:
#convert entries to int
for entry in final_train_x_list:
    for i in range(len(entry)):
        entry[i] = int(entry[i])
for entry in final_test_x_list:
    for i in range(len(entry)):
        entry[i] = int(entry[i])

In [ ]:


print(final_train_x_list[0])
print(final_test_x_list[0])

In [ ]:
final_train_y_list = []
final_test_y_list = []
for value in train_y.values:
    final_train_y_list.append(value)
for value in test_y.values:
    final_test_y_list.append(value)

In [ ]:
final_test_y_list
final_train_y_list

In [ ]:

input_dim = x_train.shape[1] # Number of features

In [ ]:
clear_session()

In [ ]:
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy

In [ ]:
def model_builder(hp):
  model = Sequential()
  #model.add(Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(Dense(units=hp_units, activation='relu'))
  model.add(Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss=SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
import keras_tuner as kt

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=100,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
import tensorflow as tf
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [ ]:
tuner.search(final_train_x_list, final_train_y_list, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
clear_session()


In [ ]:
from tensorflow import keras


model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(4))


model.add(layers.Dense(1, activation='sigmoid'))


#model.compile(loss = 'binary_crossentropy', optimizer='rmsprop' metrics = ['accuracy'], )
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])


In [ ]:
history = model.fit(final_train_x_list, final_train_y_list, epochs = 50, verbose = False, validation_data = (test_x, test_y), batch_size = 10)